In [1]:
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import theano
import theano.tensor as tt
import os
import numpy as np

In [8]:
os.chdir('/Users/lukefitzpatrick/Desktop/btsm')
from src import BayesTSM

In [9]:
df = pd.read_csv('data/bikemiles.csv')
df.head()

,y,Date,y_scaled,t
0,9191,1/1/2015,0.048411,0.000000
1,18222,1/2/2015,0.095980,0.000784
2,6282,1/3/2015,0.033089,0.001567
3,11700,1/4/2015,0.061627,0.002351
4,20022,1/5/2015,0.105461,0.003135


In [10]:
import importlib
BayesTSM = importlib.reload(BayesTSM)

In [11]:
bmod = BayesTSM.btsm(data = df, changepoint_range=1)

In [13]:
g = bmod.fit_linear_model()

/Users/lukefitzpatrick/anaconda3/lib/python3.6/site-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = 341.82, ||grad|| = 83.15: 100%|██████████| 785/785 [00:00<00:00, 913.81it/s]  
Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, m, delta, k]
Sampling 4 chains: 100%|██████████| 2800/2800 [00:48<00:00, 58.24draws/s]
/Users/lukefitzpatrick/anaconda3/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:1044: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be inter

In [12]:
s = bmod.fit_seasonal_model()

/Users/lukefitzpatrick/anaconda3/lib/python3.6/site-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = -1,435.1, ||grad|| = 982.87: 100%|██████████| 10/10 [00:00<00:00, 1127.23it/s]
Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta_weekly, beta_yearly]
Sampling 4 chains: 100%|██████████| 2800/2800 [00:04<00:00, 624.30draws/s] 
/Users/lukefitzpatrick/anaconda3/lib/python3.6/site-packages/mkl_fft/_numpy_fft.py:1044: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future thi

In [ ]:
# this uses a jump prior sd of 5
plt.subplots(figsize=(25,5))
plt.plot(g + s, color='green')
plt.scatter(df.index, df['y'], s=5)

In [ ]:
days = 100
future = pd.DataFrame({'ds': pd.date_range(df['ds'].min(), 
                                           df['ds'].max() + pd.Timedelta(days, 'D'), 
                        df.shape[0] + days)})
future['t'] = (future['ds'] - df['ds'].min()) / (df['ds'].max() - df['ds'].min())

In [ ]:
beta_yearly = trace['beta_yearly'].mean(0)
beta_weekly = trace['beta_weekly'].mean(0)
delta = trace['delta'].mean(0)

# scalar distributions
k = trace['k'].mean()
m = trace['m'].mean()

trend_forecast = []
lambda_ = trace['tau'].mean()